<a href="https://colab.research.google.com/github/jjcastilla94/Pr-ctica.-An-lisis-de-Sentimiento-en-Comentarios-de-Clientes-con-Python-e-IA/blob/main/Pr%C3%A1ctica_An%C3%A1lisis_de_Sentimiento_en_Comentarios_de_Clientes_con_Python_e_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Recolección de datos

In [24]:
!pip install nltk textblob vaderSentiment matplotlib plotly mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.3 MB/s eta 0:00:00


He realizado una amplia busqueda por una aplicación llamada Twitter donde he recolectado los siguientes tipos de comentarios sobre el polémico tema del VAR en el fútbol. Los he analizado y he recolectado los que me parecieron interesantes:

    "El VAR está arruinando el fútbol!!!, ya no se sabe cuándo acaba un partido."
    "Me parece que el VAR está ayudando a mejorar la justicia en los partidos."
    "El VAR es una vergüenza&&, no es justo para los equipos pequeños."
    "Me gustaría que el VAR fuera más rápido y preciso, hay demasiadas interrupciones."
    "El VAR debería ser más transparente, __siempre hay dudas sobre las decisiones."
    "Gracias al VAR, se evitaron varios errores importantes en el partido."
    "El VAR solo beneficia a los equipos grandes/\, es un desastre."
    "El uso del VAR está siendo abusivo, los árbitros no se atrevan a decidir sin revisarlo."
    "Aunque el VAR tiene sus problemas>>>>, prefiero que esté en el fútbol que dejar que los errores decidan el resultado."

In [25]:
import random

comentarios_positivos = [
    "Me parece que el VAR está ayudando a mejorar la justicia en los partidos..",
    "Gracias al VAR, se evitaron varios errores importantes en el partido.",
    "El VAR es la mejor funcion para que sea todo mas justo.",
]

comentarios_negativos = [
    "El VAR está arruinando el fútbol!!!, ya no se sabe cuándo acaba un partido.",
    "El VAR es una vergüenza&&, no es justo para los equipos pequeños.",
    "El VAR solo beneficia a los equipos grandes/\, es un desastre.",
    "El uso del VAR está siendo abusivo, los árbitros no se atrevan a decidir sin revisarlo.",

]

comentarios_neutros = [
    "Me gustaría que el VAR fuera más rápido y preciso, hay demasiadas interrupciones.",
    "El VAR debería ser más transparente, __siempre hay dudas sobre las decisiones.",
    "No tengo una opinión clara sobre el VAR, es regular.",
]

# Generar 100 comentarios aleatorios
comentarios = [
    random.choice(comentarios_positivos + comentarios_negativos + comentarios_neutros)
    for _ in range(100)
]

# Guardar en un archivo txt
with open("comentarios.txt", "w", encoding="utf-8") as f:
    for comentario in comentarios:
        f.write(comentario + "\n")

print("Archivo comentarios.txt generado con éxito.")

Archivo comentarios.txt generado con éxito.


#2. Preprocesamiento de datos

Tras haber seleccionado varios comentarios habría que limpiar los datos quitando caracteres especiales, etiquetas, urls, etc. antes de
almacenarlos en la base de datos.

    'El VAR está arruinando el fútbol ya no se sabe cuándo acaba un partido',
    'Me parece que el VAR está ayudando a mejorar la justicia en los partidos',
    'El VAR es una vergenza no es justo para los equipos pequeños',
    'Me gustaría que el VAR fuera más rápido y preciso hay demasiadas interrupciones',
    'El VAR debería ser más transparente siempre hay dudas sobre las decisiones',
    'Gracias al VAR se evitaron varios errores importantes en el partido',
    'El VAR solo beneficia a los equipos grandes es un desastre',
    'El uso del VAR está siendo abusivo los árbitros no se atrevan a decidir sin revisarlo',
    'Aunque el VAR tiene sus problemas prefiero que esté en el fútbol que dejar que los errores decidan el resultado'

In [26]:
import re

def limpiar_texto(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(r"http\S+|www\S+|https\S+", "", texto, flags=re.MULTILINE)  # Eliminar URLs
    texto = re.sub(r"\W", " ", texto)  # Eliminar caracteres especiales
    texto = re.sub(r"\s+", " ", texto).strip()  # Eliminar espacios extra
    return texto

# Leer y limpiar los comentarios
comentarios_limpios = []
with open("comentarios.txt", "r", encoding="utf-8") as f:
    for linea in f:
        comentarios_limpios.append(limpiar_texto(linea.strip()))

print(comentarios_limpios[:5])  # Ver los primeros 5 comentarios limpios

['el var está arruinando el fútbol ya no se sabe cuándo acaba un partido', 'el var es la mejor funcion para que sea todo mas justo', 'el var está arruinando el fútbol ya no se sabe cuándo acaba un partido', 'el var es una vergüenza no es justo para los equipos pequeños', 'el var es una vergüenza no es justo para los equipos pequeños']


#3. Almacenamiento de datos

Ahora hay que almacenar los comentarios obtenidos en una base de datos SQLite o MySQL.

*   Instalamos sqlLITE y lo convertimos a sql









In [27]:
import sqlite3

conn = sqlite3.connect("comentarios.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS comentarios (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    comentario TEXT
)
""")

cursor.executemany("INSERT INTO comentarios (comentario) VALUES (?)", [(c,) for c in comentarios_limpios])
conn.commit()
conn.close()

print("Comentarios almacenados en SQLite.")

Comentarios almacenados en SQLite.


In [28]:
import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect('comentarios.db')

# Crear un cursor
cursor = conn.cursor()

# Obtener el volcado de la base de datos
with open("comentarios.sql", "w") as f:
    for line in conn.iterdump():
        f.write(f"{line}\n")

# Cerrar la conexión
conn.close()

# Mostrar el archivo generado
print("El archivo comentarios.sql ha sido creado.")

El archivo comentarios.sql ha sido creado.


#4. Modelo de IA

Para determinar la polaridad de los comentarios(positivo,negativo o neutro). Voy a utilizar el proyecto Vader.

In [31]:
!pip install vaderSentiment

In [32]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def obtener_sentimiento(texto):
    puntaje = analyzer.polarity_scores(texto)["compound"]
    if puntaje >= 0.05:
        return "positivo"
    elif puntaje <= -0.05:
        return "negativo"
    else:
        return "neutro"

# Leer comentarios desde la BD y analizarlos
conn = sqlite3.connect("comentarios.db")
cursor = conn.cursor()
cursor.execute("SELECT comentario FROM comentarios")
comentarios = cursor.fetchall()

resultados = {"positivo": 0, "negativo": 0, "neutro": 0}

for comentario in comentarios:
    sentimiento = obtener_sentimiento(comentario[0])
    resultados[sentimiento] += 1

conn.close()

print("Análisis de Sentimientos:", resultados)

Análisis de Sentimientos: {'positivo': 0, 'negativo': 36, 'neutro': 64}


In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Función para analizar sentimiento
def obtener_sentimiento(texto):
    puntaje = analyzer.polarity_scores(texto)["compound"]
    if puntaje >= 0.05:
        return "positivo"
    elif puntaje <= -0.05:
        return "negativo"
    else:
        return "neutro"

# Leer comentarios desde la BD y analizarlos
import sqlite3

conn = sqlite3.connect("comentarios.db")
cursor = conn.cursor()
cursor.execute("SELECT comentario FROM comentarios")
comentarios = cursor.fetchall()

resultados = {"positivo": 0, "negativo": 0, "neutro": 0}
polaridades = []  # Para histogramas

for comentario in comentarios:
    texto = comentario[0]
    puntaje = analyzer.polarity_scores(texto)["compound"]
    polaridades.append(puntaje)
    sentimiento = obtener_sentimiento(texto)
    resultados[sentimiento] += 1

conn.close()

print("Resultados de Análisis de Sentimiento:", resultados)

Resultados de Análisis de Sentimiento: {'positivo': 0, 'negativo': 36, 'neutro': 64}


#5. Visualización de resultados

Crearemos diferentes gráficas para representar el porcentaje de comentarios positivos, negativos y neutros para mostrarlo de vcarias formas el resultado.

In [37]:
import plotly.graph_objects as go

# Colores degradados y modernos
colors = ["#00C9A7", "#FF6B6B", "#FFD93D"]  # Verde agua, rojo vibrante y dorado

fig = go.Figure(data=[go.Pie(
    labels=list(resultados.keys()),
    values=list(resultados.values()),
    marker=dict(colors=colors, line=dict(color='#000000', width=2)),  # Borde negro para más contraste
    textinfo='label+percent',
    insidetextorientation='radial',
    pull=[0.08, 0.08, 0],  # Resalta positivo y negativo
    hole=0.3  # Efecto "donut" ligero
)])

fig.update_layout(
    title_text="Sentimientos Detectados",
    title_font_size=22,
    showlegend=True,
    paper_bgcolor="#F4F4F4",  # Fondo suave
    font=dict(size=16)
)

fig.show()


In [39]:
import plotly.graph_objects as go

# Definir colores vibrantes
colors = {"positivo": "#27AE60", "negativo": "#C0392B", "neutro": "#2980B9"}  # Verde, rojo y azul modernos

fig = go.Figure()

# Agregar las barras con efectos 3D y brillo
for sentimiento, valor in resultados.items():
    fig.add_trace(go.Bar(
        x=[sentimiento],
        y=[valor],
        text=[valor],
        textposition='outside',
        marker=dict(
            color=colors[sentimiento],
            line=dict(color='black', width=2),
            opacity=0.8  # Hace las barras ligeramente traslúcidas para efecto "glow"
        ),
        name=sentimiento
    ))

# Configurar la animación
fig.update_layout(
    title="🔥 Distribución de Sentimientos 🔥",
    xaxis_title="Tipo de Sentimiento",
    yaxis_title="Frecuencia",
    title_font_size=24,
    paper_bgcolor="#1E1E1E",  # Fondo oscuro para más impacto
    plot_bgcolor="#2C2C2C",  # Fondo de la gráfica también oscuro
    font=dict(size=16, color="white"),
    showlegend=False,  # Oculta leyenda para un diseño más limpio
    bargap=0.3,  # Espaciado entre barras
    updatemenus=[dict(  # Botón para animación
        type="buttons",
        showactive=False,
        buttons=[dict(label="Animar",
                      method="animate",
                      args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}])]
    )]
)

# Agregar efecto de animación
fig.update_traces(marker=dict(line=dict(width=2)))
fig.show()


In [40]:
import plotly.graph_objects as go

# Definir colores vibrantes
colors = {"positivo": "#27AE60", "negativo": "#C0392B", "neutro": "#2980B9"}  # Verde, rojo y azul modernos

# Crear la figura
fig = go.Figure()

# Agregar la línea con puntos destacados
fig.add_trace(go.Scatter(
    x=list(resultados.keys()),
    y=list(resultados.values()),
    mode="lines+markers",  # Línea con puntos marcados
    line=dict(width=3, color="#F39C12"),  # Línea dorada vibrante
    marker=dict(size=10, color=[colors[s] for s in resultados.keys()], line=dict(color="black", width=2)),  # Puntos grandes con borde negro
    text=[f"{v} ocurrencias" for v in resultados.values()],
    hoverinfo="text+name"
))

# Configurar animación y diseño
fig.update_layout(
    title="📈 Evolución de Sentimientos 📈",
    xaxis_title="Tipo de Sentimiento",
    yaxis_title="Frecuencia",
    title_font_size=24,
    paper_bgcolor="#1E1E1E",  # Fondo oscuro
    plot_bgcolor="#2C2C2C",  # Fondo de la gráfica oscuro
    font=dict(size=16, color="white"),
    showlegend=False,
    xaxis=dict(showgrid=False),  # Oculta las líneas de la cuadrícula en X
    yaxis=dict(showgrid=True, gridcolor="gray"),  # Líneas sutiles en el eje Y
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(label="Animar",
                      method="animate",
                      args=[None, {"frame": {"duration": 700, "redraw": True}, "fromcurrent": True}])]
    )]
)

# Mostrar gráfico
fig.show()


#6. Ampliación

Para realizar una aplicación web en Flask que implemente toda la funcionalidad descrita, necesitamos integrar varios componentes:



1.   Flask para la creación de la web.
2.   Transformers (HuggingFace) para el análisis de sentimientos.
3.   Plotly para la visualización interactiva.
4.   MySQL para almacenar los comentarios.

Esta todo comprimido en una carpeta en la entrega.